---
toc: true
comments: true
layout: notebook
title: User Profile JavaScript/HTML Lesson
description: The College Board testing language is Pseudo Code.  Pseudo mean kind-of language that we will compare to Python.
courses: { csp: {week: 19} }
type: hacks
---

## Update/Delete User Info
> In this section we will be explaining how data for the user would be updated and deleted in the Java Springboot backend.

First, make sure you have this repository cloned so that you can see the code working in real time: `git clone https://github.com/CSA-AI/CSA_AI_lessonBackend.git`

If your frontend local server address is different from `http://localhost:4000` or `http://127.0.0.1:4000`, make sure to change it accordingly in the `SecurityConfig.java` and `MvcConfig.java` files and HTML files provided in the lesson.

### Update Through API Requests
> This is how the user would update data specific to their profile using a PUT request which communicates to the Springboot server.

- `@PutMapping` - maps to which site extension the update will be performed
- `ResponseEntity` - this is the method signature and specifies the required parameters (email, password, name)
- `findbyEmail` - here we find the person using the JPA repository by accessing the person object that matches the email requested, creating a copy of this object
- `setPassword`, `setName` - this is a JPA set function which allows us to set the new data that is updated
- `save()` - this saves the newly created person object, overwriting the existing person object with a matching email

This is essentially copy and paste, copying the pre-existing user data and pasting the new data in it's place.

In [ ]:
// mapping request
@PutMapping("/update")
    // method signature
    public ResponseEntity<Object> putPerson(@RequestParam("email") String email, @RequestParam("password") String password, @RequestParam("name") String name ) 
    {
        // copying person object
        Person person = repository.findByEmail(email);
        // saving new data to the copied person object
        person.setPassword(password);
        person.setName(name);
        // saving the copied person object with new data over the pre-existing person object with a matching email
        repository.save(person);
        // returning response from server to frontend
        return new ResponseEntity<>(email +" is updated successfully", HttpStatus.OK);
    }

In this example of HTML, we use fetch to PUT the data into the backend using parameters.

- `URLSearchParams();` - this is how we add parameters into the request towards the backend, allowing for the user to edit a pre-existing person in the backend database

In [ ]:
%%html
<body>
    <h2>Password Update</h2>
    <label for="email">Email:</label>
    <input type="email" id="email" name="email" value="toby@gmail.com" readonly><br>
    <label for="password">New Password:</label>
    <input type="password" id="password" name="password" value="test@123"><br>
    <label for="name">New Name:</label>
    <input type="text" id="name" name="name" value="Test Test"><br>
    <button onclick="updatePassword()">Update</button>
    <p id="updateMessage"></p>
    <script>
        function updatePassword() {
            const url = 'http://localhost:8085/api/person/update'; // sending to the backend API (change this address if your backend is different)
            const email = document.getElementById("email").value;
            const newPassword = document.getElementById("password").value;
            const newName = document.getElementById("name").value;
            const params = new URLSearchParams(); // creating the parameters sent to the backend
            params.append('email', email);
            params.append('password', newPassword);
            params.append('name', newName);
            fetch(url, {
                method: 'PUT', // the PUT method
                body: params,
            })
                .then(response => response.json()) // receiving the response from the backend server
                .then(data => {
                    // success
                    const updateMessage = "Updating email to: " + data.email;
                    document.getElementById("updateMessage").innerHTML = updateMessage;
                })
                // error checking
                .catch(error => console.error('Error:', error));
        }
    </script>
</body>

<body>
    <h2>Password Update</h2>
    <label for="email">Email:</label>
    <input type="email" id="email" name="email" value="toby@gmail.com" readonly><br>
    <label for="password">New Password:</label>
    <input type="password" id="password" name="password" value="test@123"><br>
    <label for="name">New Name:</label>
    <input type="text" id="name" name="name" value="Test Test"><br>
    <button onclick="updatePassword()">Update</button>
    <p id="updateMessage"></p>
    <script>
        function updatePassword() {
            const url = 'http://localhost:8085/api/person/update'; // sending to the backend API (change this address if your backend is different)
            const email = document.getElementById("email").value;
            const newPassword = document.getElementById("password").value;
            const newName = document.getElementById("name").value;
            const params = new URLSearchParams(); // creating the parameters sent to the backend
            params.append('email', email);
            params.append('password', newPassword);
            params.append('name', newName);
            fetch(url, {
                method: 'PUT', // the PUT method
                body: params,
            })
                .then(response => response.json()) // receiving the response from the backend server
                .then(data => {
                    // success
                    const updateMessage = "Updating email to: " + data.email;
                    document.getElementById("updateMessage").innerHTML = updateMessage;
                })
                // error checking
                .catch(error => console.error('Error:', error));
        }
    </script>
</body>

### Delete Through API Requests
> This is how a user with admin permission would be able to request to delete specific users in the database.

- `@DeleteMapping` - this maps the site extension through which deleting will be performed
- `ResponseEntity<>` - the method signature requesting for the email, which is later used
- `findByEmail()` - finds the user object with a matching email to the one requested using the JPA repo
- `if (person != null)` - this just checks if the user exists
- `deleteByEmail(email)` - this is another JPA function that is used to delete the person object that has the same email as the one requested
- `ResponseEntity<>(HttpStatus.NOT_FOUND)` - returns a specific error that states that the user is not found

Overall, this find the user by the email and deletes them from the database if they exist.

In [ ]:
@Transactional // method is a database operation
@DeleteMapping("/delete") // mapping request
// method signature
public ResponseEntity<Person> deletePerson(@RequestParam("email") String email) {
    // finding person object by email
    Person person = repository.findByEmail(email);
    // checks if person object requested exists
    if (person != null) {
        repository.deleteByEmail(email);
        // returns success
        return new ResponseEntity<>(person, HttpStatus.OK);
    }
    // returns error
    return new ResponseEntity<>(HttpStatus.NOT_FOUND);
}

This is similar to the update HTML, except now we are using a DELETE request to the backend, again using parameters which are sent.

In [ ]:
%%html
<body>
    <h2>User Delete</h2>
    <label for="deleteEmail">Email:</label>
    <input type="email" id="deleteEmail" name="deleteEmail"><br>
    <button onclick="deleteUser()">Delete</button>
    <p id="updateMessage2"></p>
    <script>
        function deleteUser() {
            const url = 'http://localhost:8085/api/person/delete';
            const email = document.getElementById("deleteEmail").value;
            const params = new URLSearchParams(); // creating the parameters sent to the backend
            params.append('email', email);
            fetch(url, {
                method: 'DELETE',
                body: params,
            })
                .then(response => response.json())
                .then(data => {
                    const deleteMessage = "User deleted. Email: " + data.email;
                    document.getElementById("updateMessage2").innerHTML = deleteMessage;
                })
                .catch(error => console.error('Error:', error));
        }
    </script>
</body>


<body>
    <h2>User Delete</h2>
    <label for="deleteEmail">Email:</label>
    <input type="email" id="deleteEmail" name="deleteEmail"><br>
    <button onclick="deleteUser()">Delete</button>
    <p id="updateMessage2"></p>
    <script>
        function deleteUser() {
            const url = 'http://localhost:8085/api/person/delete';
            const email = document.getElementById("deleteEmail").value;
            const params = new URLSearchParams(); // creating the parameters sent to the backend
            params.append('email', email);
            fetch(url, {
                method: 'DELETE',
                body: params,
            })
                .then(response => response.json())
                .then(data => {
                    const deleteMessage = "User deleted. Email: " + data.email;
                    document.getElementById("updateMessage2").innerHTML = deleteMessage;
                })
                .catch(error => console.error('Error:', error));
        }
    </script>
</body>

If you would like to change which users are able to update or delete other users data or make the updating or deleting of data user-specific you can change the `SecurityConfig.java` file as shown below.

In [ ]:
.authorizeHttpRequests(auth -> auth
    .requestMatchers("/authenticate", "/mvc/person/post/**").permitAll()
    .requestMatchers("/mvc/person/update/**", "/mvc/person/delete/**").hasAnyAuthority("ROLE_ADMIN") // this would be used for editing locally within the server's frontend
    .requestMatchers("/api/person/post/**", "/api/person/delete/**").hasAnyAuthority("ROLE_ADMIN") // allowing only admin to essentially access these pages from requests
    .requestMatchers("/**").permitAll()
)